# Data-driven nonlinear manifolds

## A general representation learning problem

In this notebook we demonstrate the construction of data-driven nonlinear manifolds for model reduction purposes. We learn the manifold through the identification of low-order, polynomial structure in the data through a general representation representation learning problem.<sup>[1](https://doi.org/10.48550/arXiv.2306.13748)</sup> In particular, we seek to build approximations of the form

$$
\mathbf{s}_j \approx \mathbf{s}_\text{ref} + \mathbf{V} \hat{\mathbf{s}}_j + \overline{\mathbf{V}} \boldsymbol{\Xi} \mathbf{g}(\hat{\mathbf{s}}_j), \quad \text{for } j=1,\dots,k,
$$

where $\mathbf{s}_j \in \mathbb{R}^{n}$ is the $j$th data sample, $\mathbf{s}_\text{ref} \in \mathbb{R}^{n}$ is a given reference state, $\mathbf{V} \in \mathbb{R}^{n \times r}$ and $\overline{\mathbf{V}} \in \mathbb{R}^{n \times q}$ are basis matrices, $\hat{\mathbf{s}}_j \in \mathbb{R}^{n \times r}$ is the reduced-state representation of the $j$th snapshot, and $\boldsymbol{\Xi} \in \mathbb{R}^{q \times (p-1)r}$ is the coefficient matrix. The total number of data samples is given by $k$. The vector $\mathbf{g}(\hat{\mathbf{s}}(t)) \in \mathbb{R}^{(p-1)r}$ has the form

$$
\mathbf{g}(\hat{\mathbf{s}}(t)) = \begin{pmatrix} \hat{\mathbf{s}}^2(t) \\
\hat{\mathbf{s}}^3(t) \\
\vdots                \\
\hat{\mathbf{s}}^p(t) 
\end{pmatrix},
\label{eq:polynomial}
$$

where each $\hat{\mathbf{s}}^j(t) \in \mathbb{R}^r$ consists of the $j$th power of the components of $\hat{\mathbf{s}}(t)$, that is, $\hat{\mathbf{s}}^j(t) = [\hat{s}_1(t)^j,\hat{s}_2(t)^j,\dotsc,\hat{s}_r(t)^j]$.

## Problem setup & numerical implementation

First, we import the needed things from NumPy, Matplotlib and SciPy.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

We generate the data by sampling the vector $\mathbf{s}(x,y)=\left(x,y,\sin(x)\cos(y)\right)^\top \in \mathbb{R}^3$ for $x,y \in [0,4]$. For centering the data we define the reference state $\mathbf{s}_\text{ref}$ as the column-averaged mean of the training data.

In [ ]:
# create mesh grid
nx, ny = (41, 41)
num_snapshots = nx * ny
x = np.linspace(0, 4, nx)
y = np.linspace(0, 4, ny)
xx, yy = np.meshgrid(x, y)

# compute data matrix
fxy = np.sin(xx) * np.cos(yy)
S = np.reshape([xx, yy, fxy], (3, num_snapshots))
sref = np.mean(S, axis=1)
Sref = np.array([sref,] * num_snapshots).T

Choose the model parameters:

In [ ]:
p = 3           # degree of polynomial embeddings
r = 2           # number of basis vectors in Vr
q = 1           # number of basis vectors in Vbar
tol = 1e-3      # tolerence for alternating minimization
gamma = 0       # regularization parameter
max_iter = 100  # maximum number of iterations

We now create a number of useful function handles for quickly evaluating the function $\mathbf{g}(\hat{\mathbf{s}})$ and computing the relative state error:

In [ ]:
def polynomial_form(x):
    """Polynomial representation of the reduced state snapshot data."""
    return [x**degree for degree in range(2, p+1)]


def relative_error(S_exact, S_reconstructed):
    """Calculate the relative squared Frobenius-norm error."""
    return np.linalg.norm(S_exact - S_reconstructed, 'fro') / np.linalg.norm(S_exact - Sref, 'fro')

We also create a function for the visualizuation of the reconstructed data:

In [ ]:
def plot_reconstruction(Gamma):
    """Plot the original data (fxy) and the reconstruction (fxy_Gamma)."""
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')

    fxy_Gamma = np.reshape(Gamma[2,::], (nx, ny))
    ax.plot_surface(xx, yy, fxy, color='gray', edgecolor='black', linewidth=0.1, alpha=0.2)
    surf = ax.plot_surface(xx, yy, fxy_Gamma, cmap='seismic', vmin=-1, vmax=1, alpha=0.8)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$f(x,y)$')
    ax.set_xlim(0, 4)
    ax.set_ylim(0, 4)
    ax.set_zlim(-1, 1)
    ax.view_init(30, 130)
    fig.colorbar(surf, shrink=0.5, aspect=20, pad=0.1)

    plt.show()

## Comparison of representation learning methods

### 1. Traditional POD

We start with a simple POD method. We first compute the left singular vectors of the shifted data matrix. A two-dimensional representation of the data is then obtained by means of a projection onto the two-dimensional POD subspace. The corresponding approximation of the $j$th snapshot given by

$$
\mathbf{s}_j \approx \mathbf{s}_\text{ref} + \mathbf{V}\hat{\mathbf{s}}_j, \quad \text{for } j=1,\dots,k.
$$

In [ ]:
U, ss, _ = np.linalg.svd(S - Sref, full_matrices=False)
Vr = U[:, :r]                   # select r leading singular vectors
Shat = Vr.T @ (S-Sref)          # represent data in POD coordinates

Gamma_POD = Sref + Vr @ Shat    # compute the projection of the original dataset
print(f"\nReconstruction error: {relative_error(S, Gamma_POD):.4%}")

We can now plot the reconstructions obtained using POD:

In [ ]:
plot_reconstruction(Gamma_POD)

### 2. POD-based nonlinear manifold learning

In the POD-based manifold learning approach we choose the columns of $\mathbf{V}$ to be, as is the case in the traditional POD approach, the dominant two left singular vectors of the shifted data. The third singular vector now be contained in $\overline{\mathbf{V}}$. The coefficient matrix $\boldsymbol{\Xi} \in \mathbb{R}^{q \times (p-1)r}$  is computed via the normal equations.

In [ ]:
Vbar = U[:, r:r+q]
Proj_error = S - Sref - (Vr @ Shat)
Poly = np.concatenate(polynomial_form(Shat), axis=0)
Xi = Vbar.T @ Proj_error @ Poly.T @ np.linalg.inv(Poly @ Poly.T + gamma * np.identity((p - 1)*r))

Gamma_MPOD = Sref + (Vr @ Shat) + (Vbar @ Xi @ Poly)
print(f"\nReconstruction error: {relative_error(S, Gamma_MPOD):.4%}")

Plotting the reconstructed manifold using the nonlinear manifold based POD formulation:

In [ ]:
plot_reconstruction(Gamma_MPOD)

### 3. Alternating mimization based nonlinear manifold learning

The objective is successively minimized for the three blocks of variables in turn $-$first $\boldsymbol{\Omega} = (\mathbf{V}, \overline{\mathbf{V}} )$, then $\boldsymbol{\Xi}$, then $\hat{\mathbf{S}}$$-$ with the pattern repeating until a convergence criterion is satisfied. The minimization  with respect to $\boldsymbol{\Omega}$ is a standard problem known as the orthogonal Procrustes problem. The minimization with respect to $\boldsymbol{\Xi}$ is a linear least squares problem, as in above approach. The minimization with respect to $\hat{\mathbf{S}}$ decomposes into $k$ separate problems, each of which has a single reduced  state $\hat{\mathbf{s}}_j$ as its variable. 

In [ ]:
def representation_learning_obj(x):
    """Objective function for the nonlinear regression problem in the alternating minimization approach."""
    return S[:, snapshot] - sref - (Vr @ x) - (Vbar @ Xi @ np.concatenate(polynomial_form(x), axis=0))


nrg_old = 0
print("***Starting alternating minimizations:")

# start iterations
for niter in range(max_iter):

    # step 1 - orthogonal Procrustes (update basis vectors)
    Um, _ , Vm = np.linalg.svd((S - Sref) @ np.concatenate([Shat, Xi @ Poly]).T, full_matrices=False)
    Omega = Um @ Vm
    Vr, Vbar = Omega[:, :r], Omega[:, r:r+q]

    # step 2 - linear regression (update coefficient matrix)
    Proj_error = S - Sref - (Vr @ Shat)
    rhs = np.linalg.inv(Poly @ Poly.T + (gamma * np.identity((p - 1)*r)))
    Xi = Vbar.T @ Proj_error @ Poly.T @ rhs

    # step 3 - nonlinear regression (update reduced state representation)
    for snapshot in range(num_snapshots):
        Shat[:, snapshot] = opt.least_squares(representation_learning_obj, Shat[:, snapshot], ftol=1e-9).x
    Poly = np.concatenate(polynomial_form(Shat), axis=0)

    # evaluate convergence criterion
    energy = np.linalg.norm(Vr @ Shat + (Vbar @ Xi @ Poly), 'fro')**2 / np.linalg.norm(S - Sref, 'fro')**2
    diff = abs(energy - nrg_old)
    print(f"\titeration: {niter+1:d}\tsnapshot energy: {energy:e}\t diff: {diff:e}")
    if diff < tol:
        print("***Convergence criterion active!")
        break
    nrg_old = energy # update old energy metric


Gamma_MAM = Sref + (Vr @ Shat) + (Vbar @ Xi @ Poly)
print(f"\nReconstruction error: {relative_error(S, Gamma_MAM):.4%}")

Plotting the reconstructed manifold using the alternating mimization formulation:

In [ ]:
plot_reconstruction(Gamma_MAM)

## Useful references

  1. Geelen, R., Balzano, L. and Willcox, K., 2023. Learning latent representations in high-dimensional state spaces using polynomial manifold constructions. arXiv preprint arXiv:2306.13748. [doi.org/10.48550/arXiv.2306.13748](https://doi.org/10.48550/arXiv.2306.13748)
  2. Geelen, R., Balzano, L., Wright, S. and Willcox, K., 2023. Learning physics-based reduced-order models from data using nonlinear manifolds.
  3. Geelen, R., Wright, S. and Willcox, K., 2023. Operator inference for non-intrusive model reduction with quadratic manifolds. Computer Methods in Applied Mechanics and Engineering, 403, p.115717. [doi.org/10.1016/j.cma.2022.115717](https://doi.org/10.1016/j.cma.2022.115717)